In [1]:
import pandas as pd
from catboost import CatBoostRegressor, Pool
import joblib

In [7]:
def predict_salaries(input_csv_path, model_path, output_csv_path):
    # Загрузка обученной модели
    model = joblib.load(model_path)

    # Загрузка данных из CSV файла
    data = pd.read_csv(input_csv_path, low_memory=False)
    data = data.fillna("")

    # Отбор нужных столбцов
    data = data[['id', 'academic_degree', 'accommodation_capability', 'additional_requirements',
                 'busy_type', 'career_perspective', 'education', 'education_speciality', 
                 'is_mobility_program', 'need_medcard', 'other_vacancy_benefit', 
                 'position_requirements', 'position_responsibilities', 'regionName', 
                 'regionNameTerm', 'company_business_size', 'required_certificates', 
                 'required_drive_license', 'required_experience',  'schedule_type', 'professionalSphereName', 
                 'languageKnowledge', 'hardSkills', 'softSkills']]

    # Удаляем ненужные столбцы для предсказания
    useless_columns = ['id', 'salary']  # Убедитесь, что 'salary' удален, если он уже есть
    X = data.drop(columns=useless_columns, errors='ignore')

    # Определяем категориальные признаки
    categorical_features = [col for col in X.columns if X[col].dtype == 'object']

    # Обработка категориальных данных
    for feature in categorical_features:
        X[feature] = X[feature].astype(str).fillna('missing')

    # Преобразуем все числовые колонки в числовой формат, заменяя пропуски
    for col in X.columns:
        if col not in categorical_features:
            X[col] = pd.to_numeric(X[col], errors='coerce').fillna(-1)

    # Предсказание заработной платы
    y_pred = model.predict(X)

    # Добавляем предсказанные значения в новый столбец "salary"
    data['salary'] = y_pred

    # Сохранение обновленного набора данных в новый CSV файл
    data.to_csv(output_csv_path, index=False)

In [8]:
predict_salaries("TEST_SAL.csv", "catboost_model.joblib", "sample_sumbision.csv")